# 3. Three-dimensional analysis
The objective of this final notebook is to familiarise ourselves with the three-dimensional or spectro-morphological analysis.

In [ ]:
# - basic dependencies
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from regions import PointSkyRegion, CircleSkyRegion
import matplotlib.pyplot as plt

# - Gammapy dependencies
from gammapy.data import FixedPointingInfo, Observation, observatory_locations
from gammapy.datasets import MapDataset
from gammapy.irf import load_irf_dict_from_file
from gammapy.makers import MapDatasetMaker, SafeMaskMaker
from gammapy.maps import Map, MapAxis, WcsGeom
from gammapy.modeling import Fit
from gammapy.modeling.models import (
    Models,
    SkyModel,
    FoVBackgroundModel,
    PointSpatialModel,
    GaussianSpatialModel,
    ShellSpatialModel,
    PowerLawSpectralModel,
    ExpCutoffPowerLawSpectralModel,
)

## 3.1. Data simulation

Let us simulate the data ourselves. Let us use consider three sources: a point-like source at the galactic centre, a source with Gaussian extension right beside it, and let us make both of them sit on top of a larger soure with a shell morphology.

In [ ]:
# - source 1
spectral_model_1 = ExpCutoffPowerLawSpectralModel(
    amplitue=5e-12 * u.Unit("TeV-1 cm-2 s-1"),
    index=1.5,
    reference=1 * u.TeV,
    lambda_=1 / (3 * u.TeV),  # a cutoff at 10 TeV
)
spatial_model_1 = PointSpatialModel(lon_0="0 deg", lat_0="0 deg", frame="galactic")
model_1 = SkyModel(spectral_model_1, spatial_model_1, name="point source")

# - source 2
spectral_model_2 = PowerLawSpectralModel(
    amplitude=3e-12 * u.Unit("TeV-1 cm-2 s-1"), index=2, reference=1 * u.TeV
)
spatial_model_2 = GaussianSpatialModel(
    lon_0="0.4 deg", lat_0="0.15 deg", sigma=0.2 * u.deg, frame="galactic"
)
model_2 = SkyModel(spectral_model_2, spatial_model_2, name="gaussian")

# - source 3
spectral_model_3 = PowerLawSpectralModel(
    amplitude=1e-11 * u.Unit("TeV-1 cm-2 s-1"), index=2.7, reference=1 * u.TeV
)
spatial_model_3 = ShellSpatialModel(
    lon_0="0.06 deg",
    lat_0="0.6 deg",
    radius=0.6 * u.deg,
    width=0.3 * u.deg,
    frame="galactic",
)
model_3 = SkyModel(spectral_model_3, spatial_model_3, name="shell")

# let us add a Background as well!
bkg_model = FoVBackgroundModel(dataset_name="dataset-simu")

models_simu = Models([model_1, model_2, model_3, bkg_model])

In [ ]:
# let us make a quick plot of the spectrum of the three sources
energy_bounds = [10 * u.GeV, 100 * u.TeV]
model_1.spectral_model.plot(energy_bounds, sed_type="e2dnde", label="point source")
model_2.spectral_model.plot(energy_bounds, sed_type="e2dnde", label="gaussian")
model_3.spectral_model.plot(energy_bounds, sed_type="e2dnde", label="shell")
plt.legend()
plt.show()

For the simulation we will not simulate an actual **observation**, that is a file with event list and IRF, we will rather simulate a 3D dataset. I.e. the data-cube we discussed in the previous section. To do so, we will adopt the IRF of CTA southern array.

In [ ]:
# Loading IRFs
irfs = load_irf_dict_from_file(
    "$GAMMAPY_DATA/cta-1dc/caldb/data/cta/1dc/bcf/South_z20_50h/irf_file.fits"
)

# Define the observation parameters (typically the observation duration and the pointing position):
livetime = 4.0 * u.hr
pointing_position = SkyCoord(0, 0, unit="deg", frame="galactic")
# We want to simulate an observation pointing at a fixed position in the sky.
# For this, we use the `FixedPointingInfo` class
pointing = FixedPointingInfo(
    fixed_icrs=pointing_position.icrs,
)

# Define map geometry for binned simulation
energy_reco = MapAxis.from_edges(
    np.logspace(-1, 2, 15), unit="TeV", name="energy", interp="log"
)
geom = WcsGeom.create(
    skydir=(0, 0),
    binsz=0.02,
    width=(6, 6),
    frame="galactic",
    axes=[energy_reco],
)
# It is usually useful to have a separate binning for the true energy axis
energy_true = MapAxis.from_edges(
    np.logspace(-2, 3, 50), unit="TeV", name="energy_true", interp="log"
)

empty = MapDataset.create(geom, name="dataset-simu", energy_axis_true=energy_true)

In [ ]:
# Create an in-memory observation
location = observatory_locations["cta_south"]
obs = Observation.create(
    pointing=pointing, livetime=livetime, irfs=irfs, location=location
)
print(obs)

# Make the MapDataset
maker = MapDatasetMaker(selection=["exposure", "background", "psf", "edisp"])

maker_safe_mask = SafeMaskMaker(methods=["offset-max"], offset_max=4.0 * u.deg)

dataset = maker.run(empty, obs)
dataset = maker_safe_mask.run(dataset, obs)

# Add the model on the dataset and Poisson fluctuate
dataset.models = models_simu
dataset.fake(0)

# do a print on the dataset - there is now a counts maps
print(dataset)

Let us plot what we have simulated, we can see the contribution of the different sources in the different energy bins. Let us first examine an interactive plot and then let us make different plots in different energy bins with contours of the sources we have simulated.

In [ ]:
dataset.counts.smooth(0.05 * u.deg).plot_interactive(add_cbar=True, stretch="linear")
plt.show()

**Question**: can you explain why the different sources appear and disappear in the different energy bins?

In [ ]:
energy_edges = dataset.counts.geom.axes["energy"].edges

for idx in (1, 5, 10):

    counts_map = dataset.counts.slice_by_idx({"energy": idx})
    wcs = counts_map.geom.wcs

    ax = counts_map.smooth(0.05 * u.deg).plot(add_cbar=True, stretch="linear")
    region_source_1 = spatial_model_1.to_region()
    region_source_1.to_pixel(wcs).plot(
        ax=ax, color="k", marker="*", markersize=14, markeredgewidth=1.5
    )

    region_source_2 = spatial_model_2.to_region()
    region_source_2.to_pixel(wcs).plot(ax=ax, color="dodgerblue", ls="--", lw=3)

    region_source_3 = spatial_model_3.to_region()
    region_source_3.to_pixel(wcs).plot(ax=ax, color="teal", ls="--", lw=3)

    e_min = energy_edges[idx]
    e_max = energy_edges[idx + 1]

    ax.set_title(f"{e_min:.2f} < E < {e_max:.2f}")
    plt.show()

## 3.2. Fitting!
Let us perform the fit now. But let us see first what happens if we do not include one of the simulated sources in our modelling.

In [ ]:
models_fit_1 = Models([model_1.copy(), model_3.copy(), bkg_model.copy()])

# We do not want to fit the background in this case, so we will freeze the parameters
models_fit_1["dataset-simu-bkg"].spectral_model.norm.frozen = True
models_fit_1["dataset-simu-bkg"].spectral_model.tilt.frozen = True

dataset.models = models_fit_1

In [ ]:
%%time
# let us fit it!
fit = Fit()
results = fit.run(datasets=[dataset])
print(results)
print(models_fit_1)

A very useful tool to examine the quality of a spectro-morphological fit is the residual map. In it we subtract, pixel by pixel, the best-fit model map and the actual data. Positive residual will hint to a source we have not accounted for in our model. Negative ones to a source that is not necessary to model our field of view.

In [ ]:
ax = dataset.plot_residuals_spatial(method="diff/sqrt(model)", vmin=-0.5, vmax=0.5)
region_source_2.to_pixel(wcs).plot(ax=ax, color="dodgerblue", ls="--", lw=3)
plt.show()

Let us plot the spectra we have fitted.

In [ ]:
fig, ax = plt.subplots()

plot_kwargs = {
    "sed_type": "e2dnde",
    "yunits": u.Unit("TeV cm-2 s-1"),
    "xunits": u.GeV,
}

energy_range = [100 * u.GeV, 10 * u.TeV]

models_fit_1[0].spectral_model.plot(
    energy_range,
    ax=ax,
    color="dimgray",
    ls="--",
    label="point source, fit",
    **plot_kwargs,
)
models_fit_1[0].spectral_model.plot_error(
    energy_range, ax=ax, facecolor="dimgray", alpha=0.4, **plot_kwargs
)
model_1.spectral_model.plot(
    energy_range,
    ax=ax,
    color="k",
    ls="-",
    label="point source, simulated",
    **plot_kwargs,
)

model_2.spectral_model.plot(
    energy_range,
    ax=ax,
    color="crimson",
    ls="-",
    label="gaussian, simulated",
    **plot_kwargs,
)

models_fit_1[1].spectral_model.plot(
    energy_range,
    ax=ax,
    color="turquoise",
    ls="--",
    label="shell, fit",
    **plot_kwargs,
)
models_fit_1[1].spectral_model.plot_error(
    energy_range, ax=ax, facecolor="turquoise", alpha=0.4, **plot_kwargs
)
model_3.spectral_model.plot(
    energy_range,
    ax=ax,
    color="teal",
    ls="-",
    label="shell, simulated",
    **plot_kwargs,
)
ax.set_xlabel(r"$E\,/\,{\rm GeV}$")
ax.set_ylabel(
    r"$E^2 {\rm d}\phi/{\rm d}E\,/\,({\rm TeV}\,{\rm cm}^{-2}\,{\rm s}^{-1})$"
)
ax.set_ylim([1e-14, 1e-10])
ax.legend()
plt.show()

The simulated spectrum of the point source is more or less within our estimation, but we got a completley wrong estimation of the spectral index of the shell. Of course, the source with Gaussian extension was not included in our model. 

Let us include now all the sources in the model to be used for the fit.

In [ ]:
models_fit_2 = Models([model_1.copy(), model_2.copy(), model_3.copy(), bkg_model.copy()])

# We do not want to fit the background in this case, so we will freeze the parameters
models_fit_2["dataset-simu-bkg"].spectral_model.norm.frozen = True
models_fit_2["dataset-simu-bkg"].spectral_model.tilt.frozen = True

# assign the model to the data set
dataset.models = models_fit_2
print(dataset.models)

In [ ]:
%%time
# fit it again!
fit = Fit()
results = fit.run(datasets=[dataset])
print(results)
print(models_fit_2)

Let us examine again the residuals.

In [ ]:
ax = dataset.plot_residuals_spatial(method="diff/sqrt(model)", vmin=-0.5, vmax=0.5)
plt.show()

Let us now compare the spectral models we have simulated with those we have fitted.

In [ ]:
fig, ax = plt.subplots()

plot_kwargs = {
    "sed_type": "e2dnde",
    "yunits": u.Unit("TeV cm-2 s-1"),
    "xunits": u.GeV,
}

energy_range = [100 * u.GeV, 10 * u.TeV]

models_fit_2[0].spectral_model.plot(
    energy_range,
    ax=ax,
    color="dimgray",
    ls="--",
    label="point source, fit",
    **plot_kwargs,
)
models_fit_2[0].spectral_model.plot_error(
    energy_range, ax=ax, facecolor="dimgray", alpha=0.4, **plot_kwargs
)
model_1.spectral_model.plot(
    energy_range,
    ax=ax,
    color="k",
    ls="-",
    label="point source, simulated",
    **plot_kwargs,
)

models_fit_2[1].spectral_model.plot(
    energy_range,
    ax=ax,
    color="darksalmon",
    ls="--",
    label="gaussian, fit",
    **plot_kwargs,
)
models_fit_2[1].spectral_model.plot_error(
    energy_range, ax=ax, facecolor="darksalmon", alpha=0.4, **plot_kwargs
)
model_2.spectral_model.plot(
    energy_range,
    ax=ax,
    color="crimson",
    ls="-",
    label="gaussian, simulated",
    **plot_kwargs,
)

models_fit_2[2].spectral_model.plot(
    energy_range,
    ax=ax,
    color="turquoise",
    ls="--",
    label="shell, fit",
    **plot_kwargs,
)
models_fit_2[2].spectral_model.plot_error(
    energy_range, ax=ax, facecolor="turquoise", alpha=0.4, **plot_kwargs
)
model_3.spectral_model.plot(
    energy_range,
    ax=ax,
    color="teal",
    ls="-",
    label="shell, simulated",
    **plot_kwargs,
)
ax.set_xlabel(r"$E\,/\,{\rm GeV}$")
ax.set_ylabel(
    r"$E^2 {\rm d}\phi/{\rm d}E\,/\,({\rm TeV}\,{\rm cm}^{-2}\,{\rm s}^{-1})$"
)
ax.set_ylim([1e-14, 1e-10])
ax.legend()
plt.show()

The spectra we fitted contain, within their uncertainty band, those we simulated.

Finally, a plot that is commonly shown in spectro-morphological analyses is an histogram of the contribution of each of the sources to the observed counts. Usually the counts are extracted from a small region containing all the sources. In this case let us examine the counts within the region marked by the outer edge of the shell.

In [ ]:
counts_region = CircleSkyRegion(
    center=spatial_model_3.to_region().center,
    radius=spatial_model_3.to_region().outer_radius,
)

spec = dataset.to_spectrum_dataset(counts_region, containment_correction=True)

# - predict the counts from the first source
so1 = SkyModel(models_fit_2[0].spectral_model)
spec.models = [so1]
npred_1 = Map.from_geom(spec.counts.geom)
npred_1.data = spec.npred_signal().data

# - predict the counts from the second source
so2 = SkyModel(models_fit_2[1].spectral_model)
spec.models = [so2]
npred_2 = Map.from_geom(spec.counts.geom)
npred_2.data = spec.npred_signal().data
npred_2.data *= (
    models_fit_2[1].spatial_model.integrate_geom(spec.counts.geom).quantity.to_value("")
)

# - predict the counts from the third source
so3 = SkyModel(models_fit_2[2].spectral_model)
spec.models = [so3]
npred_3 = Map.from_geom(spec.counts.geom)
npred_3.data = spec.npred_signal().data
npred_3.data *= (
    models_fit_2[2].spatial_model.integrate_geom(spec.counts.geom).quantity.to_value("")
)

In [ ]:
fig, ax = plt.subplots()

spec.plot_excess(color="k")

npred_1.plot_hist(label="point source")
npred_2.plot_hist(label="gaussian")
npred_3.plot_hist(label="shell")
(npred_1 + npred_2 + npred_3).plot_hist(ax=ax, label="total", lw=1.5, color="crimson")

ax.legend()
ax.set_ylim([1e-4, 1e4])
plt.show()